In [ ]:
!pip install torchaudio
!pip install datasets
!pip install transformers
!pip install -q https://github.com/tugstugi/dl-colab-notebooks/archive/colab_utils.zip

     |████████████████████████████████| 1.9MB 7.8MB/s 
     |████████████████████████████████| 245kB 8.4MB/s 
     |████████████████████████████████| 245kB 26.2MB/s 
     |████████████████████████████████| 122kB 31.0MB/s 
     |████████████████████████████████| 2.5MB 7.3MB/s 
     |████████████████████████████████| 901kB 39.6MB/s 
     |████████████████████████████████| 3.3MB 45.1MB/s 
  Found existing installation: huggingface-hub 0.0.13
    Uninstalling huggingface-hub-0.0.13:
      Successfully uninstalled huggingface-hub-0.0.13
     - 10kB 25.7MB/s


In [ ]:
from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2ForCTC
from datasets import load_dataset
import torch
import torchaudio
import librosa

In [ ]:
# load the model
processor = Wav2Vec2Processor.from_pretrained("tugstugi/wav2vec2-large-xlsr-53-mongolian")
model = Wav2Vec2ForCTC.from_pretrained("tugstugi/wav2vec2-large-xlsr-53-mongolian")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureExtractor(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (2): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (3): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (4): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,)

In [ ]:
test_dataset = load_dataset("common_voice", "mn", split="test[:2%]")

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/mn/6.1.0/bb59ce0bb532485ab64b5d488a8dd2addc3104f694e06bcd2c272dc608bb1112. Subsequent calls will reuse this data.


In [ ]:
test_dataset

Dataset({
    features: ['accent', 'age', 'client_id', 'down_votes', 'gender', 'locale', 'path', 'segment', 'sentence', 'up_votes'],
    num_rows: 37
})

In [ ]:
resampler = torchaudio.transforms.Resample(48_000, 16_000)

In [ ]:
?resampler

In [ ]:
# Preprocessing the datasets.
# We need to read the audio files as arrays
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)
inputs = processor(test_dataset["speech"][:2], sampling_rate=16_000, return_tensors="pt", padding=True)

with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)

print("Prediction:", processor.batch_decode(predicted_ids))
print("Reference:", test_dataset["sentence"][:2])

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


Prediction: ['бийв буг угаал эзсгүйд байгуулдэнээ түмүүхай сүт багаадагхюм юулаа', 'тэгэхээр нь мориныхоо хомыйг тотондог шүүрүхийх булив']
Reference: ['Иймэрхүү хоол идсээр байгаад дэндүү туранхай, сул дорой болсон байлаа.', 'Тэгэхээр нь мориныхоо амыг татан ташуур өгөхийг болив.']


In [ ]:
# imports for uploading/recording
import ipywidgets as widgets
from IPython.display import Audio, display, clear_output
from dl_colab_notebooks.audio import record_audio, upload_audio
import numpy as np
from scipy.io import wavfile

SAMPLE_RATE = 16000
# wav to text method
def wav_to_text(f='test.wav'):
  audio, _ = librosa.load('test.wav', sr=SAMPLE_RATE)
  input_dict = processor(torch.tensor(audio), return_tensors="pt", padding=True)
  logits = model(input_dict.input_values).logits
  pred_ids = torch.argmax(logits, dim=-1)[0]
  return processor.decode(pred_ids)

In [ ]:
#@markdown * Either record audio from microphone or upload audio from file (.mp3 or .wav) { run: "auto" }

record_or_upload = "Record" #@param ["Record", "Upload (.mp3 or .wav)"]
record_seconds =   10#@param {type:"number", min:1, max:10, step:1}

def _recognize(audio):
  display(Audio(audio, rate=SAMPLE_RATE, autoplay=True))
  !rm -f test.wav
  wavfile.write('test.wav', rate=SAMPLE_RATE, data=(32767*audio).astype(np.int16))

  transcription = wav_to_text()
  print('\n\nTRANSCRIPTION:\n')
  print(transcription)


def _record_audio(b):
  clear_output()
  audio = record_audio(record_seconds, sample_rate=SAMPLE_RATE)
  _recognize(audio)
def _upload_audio(b):
  clear_output()
  audio = upload_audio(sample_rate=SAMPLE_RATE)
  _recognize(audio)

if record_or_upload == "Record":
  button = widgets.Button(description="Record Speech")
  button.on_click(_record_audio)
  display(button)
else:
  _upload_audio("")

Starting recording for 10 seconds...


<IPython.core.display.Javascript object>

Finished recording!


It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.




TRANSCRIPTION:

цан уу намайг энх болд гэдэг зоны сургалтанд тавтай морил
